In [1]:
#r "nuget: xunit, 2.8.1"
using System.Threading;
using Xunit;

class Message
{
    static int messageHandled = 0;
    int id;
    public Message()
    {
        id = Thread.CurrentThread.ManagedThreadId;
    }

    public void Handle()
    {
        Interlocked.Increment(ref messageHandled);
        Assert.NotEqual(id, Thread.CurrentThread.ManagedThreadId);
    }

    public static void Check()
    {
        Assert.Equal(200, messageHandled);
    }
}

Installed Packages xunit, 2.8.1

In [3]:
using System.Collections.Concurrent;

var queue1 = new BlockingCollection<Message>(2); 
var queue2 = new BlockingCollection<Message>(2); 

//queue1.Add(new Message());
queue2.Add(new Message());

var ping = Task.Run(() => ProcessMessages(queue2, queue1));
var pong = Task.Run(() => ProcessMessages(queue1, queue2));

Task.WaitAll(ping, pong);

Message message;

if (queue1.TryTake(out message))
{
    Console.WriteLine("Took message from queue1");
}
else
{
    Console.WriteLine("Failed to take message from queue1");
}

if (queue2.TryTake(out message))
{
    Console.WriteLine("Took message from queue2");
}
else
{
    Console.WriteLine("Failed to take message from queue2");
}

static void ProcessMessages(BlockingCollection<Message> inputQueue, BlockingCollection<Message> outputQueue)
{
    for (int i = 0; i < 100; i++)
    {
        inputQueue.Take().Handle();
        outputQueue.Add(new Message());
    }
}

Failed to take message from queue1
Took message from queue2


In [4]:
Assert.Equal(0, queue1.Count);
Assert.Equal(0, queue2.Count);
Message.Check();